# Entendiendo las RNNs by Marcelo Torres Cisterna

Primero comenzamos importando los módulos necesarios. Para el ejemplo en cuestión utilizaré __PyTorch__. 

In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

![RNN](rnncell.png)

La unidad básica de una __Red Neuronal Recurrente__ se muestra en el apartado anterior. Al comienzo puede parecer bastante confuso ya que existen una serie de operaciones utilizadas para crear tanto el __Hidden State__ como el output. En las siguientes líneas iremos deglosando paso a paso las operaciones matemáticas que se muestran en la figura.

Podemos apreciar que el input tiene un índice __t__ y ésto es porque las RNN reciben __secuencias__ de datos como por ejemplo los valores en el tiempo de una acción o frases. Cada input de la secuencia a su vez genera un output para ese input y un estado denominado, __Hidden State__ que se utiliza en el siguiente input de la secuencia. La función de éste hidden state es básicamente guardar en su ___memoria___ la información anterior de la secuencia. Más adelante veremos que éste tipo de estructuras no funcionan en secuencias muy largas, pero afortunadamente hay otras estructuras que se adaptan bastante bien como las __Long Short Term Memory Cells (LSTM)__ o las __Gated Recurrent Units (GRU)__.

## Recordando Un Poco De Propiedades Matriciales

Comenzamos examinando algunos términos del esquema:
* $x_{t}$ : Input _t_ de la secuencia
* $W_{ax}$ : Matriz que multiplica al Input _t_ de la secuencia
* $W_{aa}$ : Matriz que multiplica al Hidden State anterior _t-1_
* $b_{a}$ : Bias term del estado de activación 
* $g_{1}$ : Activación 1
* $W_{ya}$ : Matriz que multiplica al Hidden State _t_ de la secuencia para generar el Output _t_
* $b_{y}$ : Bias term del estado de activación _t_
* $g_{2}$ : Activación 2
* $y_{t}$ : Output _t_ de la secuencia

Para obtener el estado de activación, se realiza la siguiente operación: 

$a^{<t>}=g(W_{aa}a^{<t-1>} + W_{ax}x^{<t>} + b_a)$

Sin embargo, matemáticamente hablando, lo anterior se puede también representar de la siguiente forma que es la cual habitualmente se muestra en muchas imágenes:

$a^{<t>}=g(W_{a}[a^{<t-1>},x^{<t>}] + b_a)$    

Ahora bien, el operador __[ , ]__ indica una concatenación vertical. Ésto quiere decir que los vectores, se insertan, uno al lado del otro. Consideremos los vectores $v^{<1>}$ y $v^{<2>}$. La operación $[v^{<1>},v^{<2>}]$ entrega lo siguiente:

In [2]:
v1 = np.random.rand(2,1)
v1

array([[0.89860195],
       [0.59288451]])

In [3]:
v2 = np.random.rand(2,1)
v2

array([[0.84848954],
       [0.87074064]])

In [4]:
v12 = np.concatenate((v1, v2), axis=0)
v12

array([[0.89860195],
       [0.59288451],
       [0.84848954],
       [0.87074064]])

Ahora bien, las matrices $W_{aa}$ y $W_{ax}$ , se concatenan en la fórmula alternativa, pero de manera __horizontal__. De esta forma consideramos la matriz $W_{1}$ y $W_{2}$ , la cual se convierte en $W_{12}$ quedando de la siguiente forma:

In [5]:
w1 = np.random.rand(2,2)
w1

array([[0.60648298, 0.64039855],
       [0.30904415, 0.68210811]])

In [6]:
w2 = np.random.rand(2,2)
w2

array([[0.50733205, 0.87109552],
       [0.51237973, 0.06587667]])

In [7]:
w12 = np.concatenate((w1 , w2) , axis = 1)
w12

array([[0.60648298, 0.64039855, 0.50733205, 0.87109552],
       [0.30904415, 0.68210811, 0.51237973, 0.06587667]])

Ahora verificamos las fórmulas para demostrar la equivalencia:

Fórmula 1 : $W_{1}v^{<1>} + W_{2}v^{<2>}$

In [8]:
formula1 = np.matmul(w1 , v1) + np.matmul(w2 , v2)
formula1 

array([[2.11363338],
       [1.17422935]])

Fórmula 2 : $W_{12}[v^{<1>},v^{<2>}]$ 

In [9]:
formula2 = np.matmul(w12, v12)
formula2

array([[2.11363338],
       [1.17422935]])

__OBS__ : omití a propósito las funciones de activación solo para demostrar la equivalencia

## Creando una Celda RNN con PyTorch

![Unfolded](unfolded.png)

Crear una celda RNN como la de la figura en PyTorch es bastante sencillo. Solamente necesitamos 3 inputs:
* __Input Length__ : El tamaño de entrada del input. Por ejemplo en el caso de las oraciones de texto sería el tamaño del embedding.
* __Hidden Length__ : El tamaño del hidden vector
* __Number of Layers__: Al igual que en un modelo denso habitual, aqui podemos tener un stack de capas de RNN y es en éste parámetro donde especificamos cuántas queremos

In [10]:
rnncel = nn.RNN(5,8,1 , batch_first = True)

Como output de la celda obtenemos dos elementos. Por una parte obtenemos el __Output__ el cual entrega el hidden state de la última capa para cada item de la secuencia. En segundo lugar está el __hidden__ el cual contiene ___Todos___ los hidden states calculados para el último item de la secuencia. 

Revisando la documentación de __PyTorch__ podemos ver qué dimensiones requieren los inputs

* __Input__: Dimensiones (tamaño del batch , tamaño de la secuencia , tamaño del input como por ejemplo longitud del embedding)
* __Hidden State__: Dimensiones (numero de capas , tamaño del batch , tamaño del vector hidden)

Consideremos como ejemplo el tensor $x$ correspondiente al embedding de una oración de dos palabras por eso tiene dimensiones (1 = un ejemplo , 2 = dos palabras , 5 = cada embedding de dimensión 5) . Recordar que __PyTorch__ funciona con __Tensors__

In [11]:
x = torch.tensor(np.ones((1,2,5)) , dtype=torch.float32)
x

tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]])

Ahora creamos el vector hidden

In [12]:
hidden = torch.tensor(np.ones((1,1,8)) * 2 , dtype = torch.float32)
hidden

tensor([[[2., 2., 2., 2., 2., 2., 2., 2.]]])

Ahora hacemos una propagación por la celda y obtenemos los outputs

In [13]:
output , new_hidden = rnncel(x , hidden)
print(f"Output :\n {output}")
print(f"New Hidden :\n {new_hidden}")

Output :
 tensor([[[-0.4620, -0.9650,  0.4580,  0.9926,  0.5424,  0.0114,  0.0017,
          -0.4429],
         [-0.1781, -0.3785,  0.5977,  0.5734,  0.8309,  0.6515,  0.7151,
          -0.5111]]], grad_fn=<TransposeBackward1>)
New Hidden :
 tensor([[[-0.1781, -0.3785,  0.5977,  0.5734,  0.8309,  0.6515,  0.7151,
          -0.5111]]], grad_fn=<StackBackward0>)


Los outputs tienen las siguientes dimensiones:

* __Output__: Dimensiones (tamaño del batch , tamaño de la secuencia , tamaño del vector hidden)
* __Hidden State__: Dimensiones (numero de capas , tamaño del batch , tamaño del vector hidden)

## Abriendo las Matemáticas dentro de la Celda

La celda nos permite obtener las matrices usadas en el cálculo. Primero extraemos manualmente el primer vector del primer input de la secuencia denotado como $x_{1}$ y hidden state inicial como $h_{0}$

In [14]:
x1 = x[0][0]
h0 = hidden[0][0]

Luego obtenemos las matrices respectivas a partir de la celda 

In [15]:
Waa1 = rnncel.weight_hh_l0
Wia1 = rnncel.weight_ih_l0
baa1 = rnncel.bias_hh_l0
bia1 = rnncel.bias_ih_l0

Realizando las multiplicaciones respectivas obtenemos el __Hidden Input 1__ para el primer elemento de la secuencia:

In [16]:
hidden_input1_item1 = torch.tanh(torch.matmul(Waa1,h0) + bia1 + torch.matmul(Wia1,x1) + baa1)
hidden_input1_item1

tensor([-0.4620, -0.9650,  0.4580,  0.9926,  0.5424,  0.0114,  0.0017, -0.4429],
       grad_fn=<TanhBackward0>)

Volvemos a realizar la operación con el segundo item de la secuencia $x_{2}$. Notar que aqui el vector hidden utilizado es el que salió del primer item (hidden_input1_item1):

In [17]:
x2 = x[0][1]

In [18]:
hidden_input2_item1 = torch.tanh(torch.matmul(Waa1,hidden_input1_item1) + bia1 + torch.matmul(Wia1,x2) + baa1)
hidden_input2_item1

tensor([-0.1781, -0.3785,  0.5977,  0.5734,  0.8309,  0.6515,  0.7151, -0.5111],
       grad_fn=<TanhBackward0>)

De ésta forma podemos ver que los outputs coinciden con los que entrega la RNN. Notar que el item __Output__ nos entrega todos los estados intermedios calculados para el último input de la secuencia, en éste caso el hidden_input_1_item1 y hidden_input_2_item1, mientras que __New Hidden__ retorna solo el último hidden vector de la secuencia

In [19]:
print(f"Output :\n {output}")
print(f"New Hidden :\n {new_hidden}")

Output :
 tensor([[[-0.4620, -0.9650,  0.4580,  0.9926,  0.5424,  0.0114,  0.0017,
          -0.4429],
         [-0.1781, -0.3785,  0.5977,  0.5734,  0.8309,  0.6515,  0.7151,
          -0.5111]]], grad_fn=<TransposeBackward1>)
New Hidden :
 tensor([[[-0.1781, -0.3785,  0.5977,  0.5734,  0.8309,  0.6515,  0.7151,
          -0.5111]]], grad_fn=<StackBackward0>)


Es importante notar que hay que añadir una capa __Densa__ dependiendo de si nuestro modelo es one-to-one , many-to-many, etc

## Creando un Stack de RNN

El ejemplo anterior explicaba de qué forma una sola celda realiza una propagación hacia adelante para una secuencia de longitud dos. Ahora tomaremos el mismo ejemplo anterior, pero colocaremos otra celda RNN sobre la anterior.

In [20]:
rnnceldoble = nn.RNN(5,8,2 , batch_first = True)

En éste caso debemos crear otro vector hidden inicial ya que ahora tenemos dos celdas en vez de una:

In [21]:
hiddendoble = torch.tensor(np.ones((2,1,8)) * 2 , dtype = torch.float32)
hiddendoble

tensor([[[2., 2., 2., 2., 2., 2., 2., 2.]],

        [[2., 2., 2., 2., 2., 2., 2., 2.]]])

In [22]:
output2 , new_hidden2 = rnnceldoble(x , hiddendoble)
print(f"Output Doble RNN :\n {output2}")
print(f"New Hidden Doble RNN :\n {new_hidden2}")

Output Doble RNN :
 tensor([[[ 0.3904,  0.7600, -0.4166,  0.8753, -0.8069,  0.9519,  0.6347,
          -0.7350],
         [-0.1631,  0.2121,  0.2217, -0.3851, -0.5885,  0.7872, -0.1114,
           0.0641]]], grad_fn=<TransposeBackward1>)
New Hidden Doble RNN :
 tensor([[[ 0.1707, -0.5239, -0.6099, -0.2825,  0.7655, -0.1127, -0.2522,
           0.3227]],

        [[-0.1631,  0.2121,  0.2217, -0.3851, -0.5885,  0.7872, -0.1114,
           0.0641]]], grad_fn=<StackBackward0>)


Obtenemos las matrices respectivas

In [23]:
Waa_layer1 = rnnceldoble.weight_hh_l0
Wia_layer1 = rnnceldoble.weight_ih_l0
baa_layer1 = rnnceldoble.bias_hh_l0
bia_layer1 = rnnceldoble.bias_ih_l0
Waa_layer2 = rnnceldoble.weight_hh_l1
Wia_layer2 = rnnceldoble.weight_ih_l1
baa_layer2 = rnnceldoble.bias_hh_l1
bia_layer2 = rnnceldoble.bias_ih_l1

Comenzamos el cálculo con el primer input de la secuencia $x_{1}$. Recordar que ahora el vector es doble (por las 2 RNN). Denotamos al vector hidden inicial como $h_{00}$

In [24]:
x1 = x[0][0]
h00 = hiddendoble[0][0]

Calculamos el primer y segundo hidden state del primer input de la secuencia

In [25]:
hidden_input1_item1_doble = torch.tanh(torch.matmul(Waa_layer1,h00) + bia_layer1 + torch.matmul(Wia_layer1,x1) + baa_layer1)
hidden_input1_item1_doble

tensor([ 0.9161, -0.9770, -0.1195,  0.9145,  0.9790, -0.8160,  0.9171, -0.2385],
       grad_fn=<TanhBackward0>)

In [26]:
# Vector inicial para la segunda RNN
h01 = hiddendoble[1][0]
h01

tensor([2., 2., 2., 2., 2., 2., 2., 2.])

In [27]:
hidden_input2_item1_doble = torch.tanh(torch.matmul(Waa_layer2,h01) + bia_layer2 + torch.matmul(Wia_layer2,hidden_input1_item1_doble) + baa_layer2)
hidden_input2_item1_doble

tensor([ 0.3904,  0.7600, -0.4166,  0.8753, -0.8069,  0.9519,  0.6347, -0.7350],
       grad_fn=<TanhBackward0>)

A continuación continuamos con el segundo input de la secuencia, denotado como $x_{2}$

In [28]:
x2 = x[0][1]

Notar que el hidden vector inicial para el input 2 es el hidden_vector_1 generado con el input 1

In [29]:
hidden_input1_item2_doble = torch.tanh(torch.matmul(Waa_layer1,hidden_input1_item1_doble) + bia_layer1 + torch.matmul(Wia_layer1,x2) + baa_layer1)
hidden_input1_item2_doble

tensor([ 0.1707, -0.5239, -0.6099, -0.2825,  0.7655, -0.1127, -0.2522,  0.3227],
       grad_fn=<TanhBackward0>)

Finalmente calculamos el output 2 del segundo input, en el cual se usan tanto los hidden states del input 1 como los generados anteriormente

In [30]:
hidden_input2_item2_doble = torch.tanh(torch.matmul(Waa_layer2,hidden_input2_item1_doble) + bia_layer2 + torch.matmul(Wia_layer2,hidden_input1_item2_doble) + baa_layer2)
hidden_input2_item2_doble

tensor([-0.1631,  0.2121,  0.2217, -0.3851, -0.5885,  0.7872, -0.1114,  0.0641],
       grad_fn=<TanhBackward0>)

De ésta forma podemos observar que tanto los outputs como los hidden states calculados manualmente coinciden con los entregados por la celda

In [31]:
output2

tensor([[[ 0.3904,  0.7600, -0.4166,  0.8753, -0.8069,  0.9519,  0.6347,
          -0.7350],
         [-0.1631,  0.2121,  0.2217, -0.3851, -0.5885,  0.7872, -0.1114,
           0.0641]]], grad_fn=<TransposeBackward1>)

In [32]:
new_hidden2

tensor([[[ 0.1707, -0.5239, -0.6099, -0.2825,  0.7655, -0.1127, -0.2522,
           0.3227]],

        [[-0.1631,  0.2121,  0.2217, -0.3851, -0.5885,  0.7872, -0.1114,
           0.0641]]], grad_fn=<StackBackward0>)